# Converting .xml to pd.DataFrame

In [1]:
#Import der packages

import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

Die Daten der Abgeordneten liegen im XML-Format vor und müssen daher zunächst für die Bearbeitung in ein DataFrame eingelesen werden. Hierzu wird eine Funktion definiert, die die einzelnen Eigenschaften eines Abgeordneten (z.B. Geburtsjahr) ausliest.

In [2]:
#Import von .xml Daten

def read_xml(path):
   xml = ET.parse(path).getroot()
   return xml

#Definieren der Funktion

def search_for_name(tree, attr):
   names = []
   for child in tree:
       if child.tag == attr:
           names += [child.text]
       else:
           names += search_for_name(child, attr)
   return names

#Test der Funktion für das Geschlecht

if __name__=="__main__":
   xml = read_xml("MDB_STAMMDATEN.XML")
   names = search_for_name(xml,"GESCHLECHT")
   
print(names[:4])
print(len(names))

['männlich', 'männlich', 'weiblich', 'weiblich']
4073


Mit der Funktion definiert, soll nun ein erstes DataFrame mit folgenden Spalten zusammengesetzt werden:

In [3]:
liste_merk = ["ID", "NACHNAME", "VORNAME","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND", "WP"]

column_list = []
for attr in liste_merk:
    if __name__=="__main__":
        xml = read_xml("MDB_STAMMDATEN.XML")
        names = search_for_name(xml, attr)
        column_list.append(names)

Es muss überprüft werden, ob die Spaltenlängen übereinstimmen. Offensichtlich gibt es noch Probleme bei den Namen und Wahlperiode Spalten. Auf sie wird daher erstmal verzichtet.

In [4]:
print(len(column_list))

for i in column_list:
    print(len(i))

9
4073
4336
4336
4073
4073
4073
4073
4073
11600


Mit den verbleibenden Spalten kann jedoch schon ein DataFrame erstellt werden. 
Hierfür wird auf die vorhin ausgelesenen Daten zurückgegriffen.

In [5]:
liste_merk_2 = ["ID","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND"]
df_first = pd.DataFrame(np.column_stack([column_list[0],column_list[3],column_list[4],column_list[5],column_list[6],column_list[7]]), columns = liste_merk_2)
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",None
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,None
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien
3,11000004,weiblich,CDU,1933,Ärztin,None
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",None


Beim Geburtsland müssen nun die None Values mit "Deutschland" ersetzt werden.

In [6]:
df_first.loc[df_first["GEBURTSLAND"].isnull(), "GEBURTSLAND"] = "Deutschland"

Nun muss der Parsing-Prozess weiterentwickelt werden, um auch die Vor- und Nachnamen korrekt einzulesen. Hierfür wird zunächst eine neue Funktion erstellt, dann damit die Daten ausgelesen und sie schlussendlich ins DataFrame eingefügt.

In [7]:
#Definieren der Funktion

def search_for_nameRoots(root):
 names = []
 for child in root:
     if child.tag == "NAMEN":
         names += [child]
     else:
         names += search_for_nameRoots(child)
 return names

xml = read_xml(path = "MDB_STAMMDATEN.XML")
nameRoots = search_for_nameRoots(xml)

#Auslesen der Daten

firstNameElements = []
for element in nameRoots:
   firstNameElements += [element[0].find('VORNAME').text]

lastNameElements = []
for element in nameRoots:
   lastNameElements += [element[0].find('NACHNAME').text]

#Einfügen ins DataFrame

df_first["NACHNAME"] = lastNameElements
df_first["VORNAME"] = firstNameElements

**Die Integration ist geglückt!**

In [8]:
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich


Zu guter Letzt müssen noch die Wahlperioden eingefügt werden. Hierfür muss eine Funktion erstellt werden, die jeder ID eine Liste aller Wahlperioden zuordnet, in denen der jeweilige Abgeordnete im Bundestag saß. Langfristig soll auf diese Perioden umindiziert werden. Zuerst wird eine Liste mit Sublisten der einzelnen Abgeordneten erstellt...

---
Zur Erklärung der Funktionsweise der Funktion:

Die nested Funktion liest alle WP-Elemente der XML-Daten aus und fügt sie einer Liste hinzu   bis der "ID"-Tag erreicht wird, der einen neuen Abgeordneten ankündigt. Dies wird 
für alle Abgeordneten durchgeführt. In der Hautpfunktion werden dann die Listen der nested Funktion als Sublisten in eine Gesamtliste eingereiht.

---

In [9]:
def search_for_wahlRoots1(root):
    
    #Beginn der nested function
    def search_for_wpRoots2(root2):
        wplist2 = []
        for child2 in root2:
            if child2.tag == "WP":
                wplist2 += [child2]
            elif child2.tag == "ID":
                break
            else:
                wplist2 += search_for_wpRoots2(child2)
        return wplist2
    #Ende der nested function
    
    wplist = []
    for child in root:
        if child.tag == "WAHLPERIODEN":
            wplist += [search_for_wpRoots2(child)]
        else:
            wplist += search_for_wahlRoots1(child)
    return wplist

WProots = search_for_wahlRoots1(xml)

Jetzt müssen den einzelnen Sublisten noch die zugehörige ID zugeordnet werden, um sie ins DataFrame einfügen zu können.

In [10]:
def extracting_WP(list1):
    WPelements = []
    for i in list1:
        sub_list = []
        for a in i:
            sub_list += [a.text]
        WPelements += [sub_list]
    return WPelements

WPs = extracting_WP(WProots)

Und einfügen ins DataFrame:

In [11]:
df_first["WAHLPERIODEN"] = WPs
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred,"[5, 6, 7, 8, 9, 10, 11]"
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst,"[3, 4, 5, 6, 7]"
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie,"[2, 3, 4]"
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else,"[11, 12]"
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich,"[15, 16, 12, 13, 14]"


Der gleiche Prozess ist äquivalent für die Mandatsarten.

In [12]:
#Definieren der Funktion und Datenauslese

def search_for_MandatsArtRoots1(root):
    
    #Beginn der nested function 
    def search_for_MandatRoots2(root2):
        mandatlist2 = []
        for child2 in root2:
            if child2.tag == "MANDATSART":
                mandatlist2 += [child2]
            elif child2.tag == "ID":
                break
            else:
                mandatlist2 += search_for_MandatRoots2(child2)
        return mandatlist2
    #Ende der nested function
    
    mandatlist = []
    for child in root:
        if child.tag == "WAHLPERIODEN":
            mandatlist += [search_for_MandatRoots2(child)]
        else:
            mandatlist += search_for_MandatsArtRoots1(child)
    return mandatlist

MandatRoots = search_for_MandatsArtRoots1(xml)
    
#Zuordung  der Sublisten zu IDs

def extracting_Mandate(list1):
    MandatElements = []
    for i in list1:
        sub_list = []
        for a in i:
            sub_list += [a.text]
        MandatElements += [sub_list]
    return MandatElements

Mandate = extracting_Mandate(MandatRoots)

Auch hier ins DataFrame eingefügt:

In [13]:
df_first["MANDATSARTEN"] = Mandate
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred,"[5, 6, 7, 8, 9, 10, 11]","[Direktwahl, Direktwahl, Direktwahl, Direktwah..."
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst,"[3, 4, 5, 6, 7]","[Landesliste, Landesliste, Landesliste, Landes..."
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie,"[2, 3, 4]","[Landesliste, Landesliste, Landesliste]"
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else,"[11, 12]","[Volkskammer, Landesliste]"
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich,"[15, 16, 12, 13, 14]","[Direktwahl, Direktwahl, Direktwahl, Direktwah..."


# Data Cleaning


Das Dataframe muss nun gecleant werden, um gut für die Visualisierung genutzt werden zu können. 

Hierzu müssen:
- Parteien sinnvoll zusammengefasst werden
- Geschlecht, ID, Namen, Geburtsjahr und Geburtsort auf Dopplungen und Korrektheit überprüft werden
- Geburtsland, Mandatsarten und Geschlecht mit Boolean Werten ersetzt werden

#### Zusammenfassen von Parteien

Erstmal schauen, welche Parteien vorhanden sind.

In [14]:
df_first['PARTEI_KURZ'].unique()

array(['CDU', 'FDP', 'SPD', 'KPD', 'DP', 'CSU', 'DZP', 'FU', 'GRÜNE',
       'BP', 'BÜNDNIS 90/DIE GRÜNEN', 'GB/ BHE', 'DIE GRÜNEN/BÜNDNIS 90',
       'PDS', 'PDS/LL', 'DIE LINKE.', 'SRP', 'Plos', None, 'WAV', 'DRP',
       'CSUS', 'DSU', 'DPB', 'CVP', 'DPS', 'AfD', 'Blaue'], dtype=object)

Glossar:

* KPD = Kommunistische Partei Deutschlands
* DP = Deutsche Partei
* DZP = Deutsche Zentrumspartei
* FU = Föderalistische Union (Fraktion aus BP und DZP)
* BP = Bayernpartei
* GB/ BHE = Gesamtdeutscher Block/Bund der Heimatvertriebenen und Entrechteten
* PDS = Partei des Demokratischen Sozialismus 
* PDS/LL = Partei des Demokratischen Sozialismus - Landesliste
* SRP = Sozialistische Reichspartei
* Plos = Parteilos
* WAV = Wirtschaftliche Aufbau-Vereinigung
* DRP = Deutsche Reichspartei
* CSUS = CSU-Saar
* DSU = Deutsche Soziale Union
* DPB = Deutsche Partei Bayern
* CVP = Christdemokratische Volkspartei des Saarlandes
* DPS = Demokratische Partei Saar (Saarpartei der FDP)
* Blaue = Blaue Partei

None Einträge müssen gefüllt oder gedropt werden

CDU, CSU, CSUS, CVP -> Union

Grüne, Bündnis 90/Die Grünen, Die Grünen/Bündnis 90 -> Bündnis 90/Die Grünen

PDS, PDS/LL, Die Linke -> Die Linke

FDP, DPS -> FDP

AfD, Blaue -> AfD

KPD, DP, DZP, FU, BP, GB/ BHE, SRP, WAV, DRP, DSU, DPB, Plos -> Andere

Nun erfolgt das Ersetzen:

In [15]:
df_first['PARTEI_KURZ'].replace(["CDU", "CSU", "CSUS", "CVP"],["Union", "Union", "Union", "Union"], inplace = True)
df_first['PARTEI_KURZ'].replace(["GRÜNE", "BÜNDNIS 90/DIE GRÜNEN", "DIE GRÜNEN/BÜNDNIS 90"],["Bündnis 90/Die Grünen", "Bündnis 90/Die Grünen", "Bündnis 90/Die Grünen"], inplace = True)
df_first['PARTEI_KURZ'].replace(["PDS", "PDS/LL", "DIE LINKE."],["Die Linke", "Die Linke", "Die Linke"], inplace = True)
df_first['PARTEI_KURZ'].replace(["FDP", "DPS"],["FDP", "FDP"], inplace = True)
df_first['PARTEI_KURZ'].replace(["AfD", "Blaue"],["AfD", "AfD"], inplace = True)
df_first['PARTEI_KURZ'].replace(["KPD", "DP", "DZP", "FU", "BP", "GB/ BHE", "SRP", "WAV", "DRP", "DSU", "DPB", "Plos"],["Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere"], inplace = True)

Die None Werte sollen mit "Andere" ersetzt werden.

In [16]:
df_first.PARTEI_KURZ.fillna(value="Andere", inplace=True)

**Das Zusammenfassen ist geglückt!**

In [17]:
df_first['PARTEI_KURZ'].unique()

array(['Union', 'FDP', 'SPD', 'Andere', 'Bündnis 90/Die Grünen',
       'Die Linke', 'AfD'], dtype=object)

#### Dopplungen überprüfen

Für das Geschlecht...

In [18]:
df_first['GESCHLECHT'].unique()

array(['männlich', 'weiblich'], dtype=object)

...und die ID

In [19]:
df_first[df_first.duplicated(subset="ID", keep=False)]

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN


...nicht zu vergessen die Abgeordneten (wo also Vor- und Nachnamen, sowie Geburtsdatum übereinstimmen).

In [20]:
df_first[df_first.duplicated(subset=["NACHNAME", "VORNAME", "GEBURTSDATUM"], keep=False)]

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN


#### Daten auf Korrektheit testen

Für das Geburtsdatum...

In [21]:
df_first['GEBURTSDATUM'].unique()

array(['1930', '1909', '1913', '1933', '1950', '1919', '1912', '1876',
       '1944', '1920', '1941', '1904', '1922', '1879', '1902', '1924',
       '1936', '1890', '1901', '1943', '1914', '1896', '1947', '1949',
       '1928', '1946', '1889', '1888', '1955', '1934', '1940', '1951',
       '1923', '1942', '1880', '1939', '1932', '1925', '1918', '1894',
       '1927', '1882', '1899', '1931', '1895', '1898', '1964', '1915',
       '1921', '1893', '1938', '1917', '1926', '1911', '1891', '1903',
       '1937', '1905', '1900', '1929', '1910', '1957', '1908', '1907',
       '1897', '1883', '1945', '1954', '1906', '1948', '1952', '1935',
       '1956', '1959', '1887', '1884', '1885', '1966', '1953', '1892',
       '1886', '1916', '1958', '1960', '1965', '1961', '1963', '1881',
       '1877', '1875', '1878', '1962', '1971', '1968', '1967', '1969',
       '1972', '1973', '1970', '1976', '1975', '1974', '1977', '1983',
       '1978', '1980', '1979', '1981', '1982', '1986', '1985', '1984',
      

...und auch die Geburtsorte

In [22]:
df_first['GEBURTSLAND'].unique()

array(['Deutschland', 'Jugoslawien', 'Australien', 'Kenia', 'Spanien',
       'CSSR', 'Ukraine', 'Finnland', 'Dänemark', 'Österreich',
       'Lettland', 'Oberösterreich',
       'Tansania (ehem. Deutsch-Ost-Afrika)', 'Niederlande',
       'Niederösterreich', 'Estland', 'ehem. Deutsch-Ost-Afrika',
       'Tschechoslowakei', 'Slowakei', 'Ceylon', 'Rumänien', 'Perú',
       'Schweiz', 'Israel', 'Ungarn', 'Frankreich', 'Tonga-Insel, Südsee',
       '(damalige) CSR', 'Tansania', 'USA', 'CSFR', 'Polen',
       'Südwestafrika', 'Türkei', 'Italien', 'Kroatien', 'Iran',
       'Russland'], dtype=object)

#### Ersetzung durch Booleans

Alle Geburtsländer, die nicht Deutschland sind, 
werden der Einfachheit halber zusammengefasst und mit 0 kodiert.

In [23]:
df_first["AUSLAND"] = df_first["GEBURTSLAND"] != "Deutschland"
df_first['AUSLAND'].replace([True, False], [1,0], inplace = True)

Ähnliche Umgestaltung auch für die Spalte mt dem Geschlecht.

In [24]:
df_first['GESCHLECHT'].replace(["männlich", "weiblich"],[1,0], inplace = True)

...und das Gleiche noch einmal für Mandatsart, wobei Volkskammer hier zu 
Landesliste zusammengefasst wird. Da die zu änderden Strings in einer Subliste liegen, muss zunächst eine Funktion erstellt werden, die diese ausliest und anschließend mit einem passendem Eintrag aus einem Dictionary ersetzt.

In [25]:
mandate = list(df_first['MANDATSARTEN'])
dictionarym = {"Direktwahl": 1, "Landesliste": 0, "Volkskammer": 0}

def replace_mandate(mandate_list, dictionary):
    mandate_list = [[dictionary.get(item, item) for item in lst] for lst in mandate_list]
    return mandate_list

mandate = replace_mandate(mandate, dictionarym)
df_first["MANDATSARTEN"] = mandate

# Engineering a DataFrame for Prediction

Nachdem das Cleaning abgeschlossen ist, muss das DataFrame noch ein wenig verändert werden, um fit für die spätere Vorhersage zu werden.

Diese Aufgaben umfassen:
- eine Boolean-Spalte für jede Wahlperiode erstellen (um nicht jedes Mal den Wert aus einer Subliste auslesen zu müssen)
- für jede Wahlperiode ein eigenes DataFrame erstellen
- die Anzahl der Abgeordneten in jeder Wahlperiode mit historischen Daten abgleichen
- auf die Wahlperioden umindizieren
- das Geburtsdatum mit dem Alter ersetzen
- die Mandatsart für jede Wahlperiode einzeln eintragen
- überflüssige Spalten droppen
- das fertige DataFrame exportieren

In [26]:
#Für den nächsten Abschnitt erstmal einen neuen DataFrame Namen festlegen
dfnew = df_first
dfnew.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,AUSLAND
0,11000001,1,Union,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred,"[5, 6, 7, 8, 9, 10, 11]","[1, 1, 1, 1, 1, 1, 1]",0
1,11000002,1,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst,"[3, 4, 5, 6, 7]","[0, 0, 0, 0, 0]",0
2,11000003,0,Union,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie,"[2, 3, 4]","[0, 0, 0]",1
3,11000004,0,Union,1933,Ärztin,Deutschland,Ackermann,Else,"[11, 12]","[0, 0]",0
4,11000005,1,Union,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich,"[15, 16, 12, 13, 14]","[1, 1, 1, 1, 1]",0


#### Umtragung von Wahlperioden in einzelne Spalten

Nun soll für jede Wahlperiode eine einzelne Spalte mit Boolean-Werten erstellt werden. Erstmal gucken, wie viele Wahlperioden es überhaupt gibt und diese in eine Liste einfügen.

In [27]:
unique_list = []
for e in dfnew["WAHLPERIODEN"]:
    for i in e:
        if int(i) in unique_list:
            None
        else:
            unique_list.append(int(i))
            
unique_list.sort()            
print(unique_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


Jetzt wird die Funktion erstellt, die die aktuelle Wahlperiodenliste ausliest, und anschließend die Booleans in die zugehörigen Spalten einträgt. 

---
Funktionsweise der Funktion:
Zuerst wird die nested function "listchecker" definiert, welche 1 bzw. 0 ausgibt, je nachdem ob ein bestimmtes Element in einer Liste enthalten ist. Anschließend iterieren wir über den Index aller Einträge unseres DataFrames (daher Range 4073) und erstellen für jede Zeile eine Liste "WPListe". Anschließend wir für jedes Element in der Wahlperioden Spalte einer jeder solchen Liste eine neue Spalte "WP" + die jeweilige Zahl erstellt. Dann wird die listchecker Funktion auf diese Liste angewendet, wobei sie 1 ausgibt, wenn die Zahl der jeweiligen Spalte auch in WPListe enthalten ist.

---

In [28]:
def WPcolumnchecker(column, range1):
    
    #Nested Funktion, die 1 oder 0 ausgibt
    def listchecker(WPNum, WPList):
        if WPNum in WPList:
            return 1
        else:
            return 0
    
    #Iteration über den gesamten Index
    for n in range1:
        WPstr = "WP" + str(n)
        emp_list = []
        nstr= str(n)
        for index in range(0,4073):
            WPListe = column.loc[index]
            emp_list.append(listchecker(nstr, WPListe))
        dfnew[WPstr] = emp_list
        
#Festsetzung der Parameter für die Funktion: Range ist die Liste aller Wahlperioden, die Spalte
#die Wahlperiodenspalte des DataFrames
dfrange1 = unique_list
dfcolumn = dfnew["WAHLPERIODEN"]

#Anwendung der Funktion auf die Wahlperiodenliste und Einfügen ins DataFrame
WPcolumnchecker(dfcolumn, dfrange1)

Überprüfen wir das Ganze einmal anhand eines Beispiels.

In [29]:
print(dfnew.loc[1])

ID                            11000002
GESCHLECHT                           1
PARTEI_KURZ                        FDP
GEBURTSDATUM                      1909
BERUF           Rechtsanwalt und Notar
GEBURTSLAND                Deutschland
NACHNAME                     Achenbach
VORNAME                          Ernst
WAHLPERIODEN           [3, 4, 5, 6, 7]
MANDATSARTEN           [0, 0, 0, 0, 0]
AUSLAND                              0
WP1                                  0
WP2                                  0
WP3                                  1
WP4                                  1
WP5                                  1
WP6                                  1
WP7                                  1
WP8                                  0
WP9                                  0
WP10                                 0
WP11                                 0
WP12                                 0
WP13                                 0
WP14                                 0
WP15                     

Für die spätere Visualisierung basteln wir nun noch für jede Wahlperiode ein einzelnes DataFrame. Da es keine Funktion zum automatischen Erstellen von DataFrames gibt, müssen wir das per Hand erledigen.

In [30]:
df_WP1 = dfnew.loc[dfnew["WP1"] == 1]
df_WP2 = dfnew.loc[dfnew["WP2"] == 1]
df_WP3 = dfnew.loc[dfnew["WP3"] == 1]
df_WP4 = dfnew.loc[dfnew["WP4"] == 1]
df_WP5 = dfnew.loc[dfnew["WP5"] == 1]
df_WP6 = dfnew.loc[dfnew["WP6"] == 1]
df_WP7 = dfnew.loc[dfnew["WP7"] == 1]
df_WP8 = dfnew.loc[dfnew["WP8"] == 1]
df_WP9 = dfnew.loc[dfnew["WP9"] == 1]
df_WP10 = dfnew.loc[dfnew["WP10"] == 1]
df_WP11 = dfnew.loc[dfnew["WP11"] == 1]
df_WP12 = dfnew.loc[dfnew["WP12"] == 1]
df_WP13 = dfnew.loc[dfnew["WP13"] == 1]
df_WP14 = dfnew.loc[dfnew["WP14"] == 1]
df_WP15 = dfnew.loc[dfnew["WP15"] == 1]
df_WP16 = dfnew.loc[dfnew["WP16"] == 1]
df_WP17 = dfnew.loc[dfnew["WP17"] == 1]
df_WP18 = dfnew.loc[dfnew["WP18"] == 1]
df_WP19 = dfnew.loc[dfnew["WP19"] == 1]

Mal ein Beispiel:

In [31]:
df_WP19.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,...,WP10,WP11,WP12,WP13,WP14,WP15,WP16,WP17,WP18,WP19
195,11000198,1,Union,1952,"Landwirtschaftsmeister, Parl. Staatssekretär",Deutschland,Bleser,Peter,"[12, 13, 14, 15, 16, 17, 18, 19]","[1, 1, 1, 1, 1, 1, 1, 1]",...,0,0,1,1,1,1,1,1,1,1
360,11000365,1,Die Linke,1950,"Autor, Geschäftsführer",Deutschland,Dehm-Desoi,Jörg-Diether,"[12, 16, 17, 18, 19]","[0, 0, 0, 0, 0]",...,0,0,1,0,0,0,1,1,1,1
609,11000616,1,Union,1952,Parl. Staatssekretär,Deutschland,Fuchtel,Hans-Joachim,"[11, 12, 13, 14, 15, 16, 17, 18, 19]","[1, 1, 1, 1, 1, 1, 1, 1, 1]",...,0,1,1,1,1,1,1,1,1,1
749,11000756,1,Die Linke,1948,Rechtsanwalt,Deutschland,Gysi,Gregor,"[11, 12, 13, 14, 16, 17, 18, 19]","[0, 1, 1, 1, 1, 1, 1, 1]",...,0,1,1,1,1,0,1,1,1,1
1014,11001023,0,Die Linke,1951,"Dipl.-Soziologin, Journalistin",Deutschland,Jelpke,Ulla,"[12, 13, 14, 16, 17, 18, 19]","[0, 0, 0, 0, 0, 0, 0]",...,0,0,1,1,1,0,1,1,1,1


#### Abgleichung der Abgeordnetenzahl mit historischen Daten

Weiter gehts damit, dass wir schauen, ob die Zahl der Abgeordneten im DataFrame auch mit den tatsächlichen historischen Daten übereinstimmen. Erst einmal beispielhaft an der letzten Wahlperiode.

In [32]:
df_WP19.describe()

,GESCHLECHT,AUSLAND,WP1,WP2,WP3,WP4,WP5,WP6,WP7,WP8,...,WP10,WP11,WP12,WP13,WP14,WP15,WP16,WP17,WP18,WP19
count,711.000000,711.000000,711.0,711.0,711.0,711.0,711.0,711.0,711.000000,711.000000,...,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.0
mean,0.690577,0.016878,0.0,0.0,0.0,0.0,0.0,0.0,0.001406,0.001406,...,0.002813,0.007032,0.021097,0.042194,0.091421,0.158931,0.232068,0.407876,0.592124,1.0
std,0.462581,0.128904,0.0,0.0,0.0,0.0,0.0,0.0,0.037503,0.037503,...,0.053000,0.083623,0.143809,0.201173,0.288409,0.365869,0.422449,0.491786,0.491786,0.0
min,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0
75%,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.0
max,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


Es gibt also scheinbar 711 Abgeordnete in der 19. Wahlperiode, obwohl es eigentlich nur 709 sein dürften. Die 2 Überschüssigen Abgeordneten müssen ausfindig gemacht werden.

**Die SPD hat nur 153 gewählte Abgeordnete, im DataFrame sind aber 155!**

In [33]:
len(df_WP19[df_WP19.PARTEI_KURZ == "SPD"])

155

Eine Überprüfung mit den historischen Daten zeigt das Problem: Die SPD hatte 2 Abgeordnete (Reimann, Kelber) die ausgeschieden sind und mit neuen (Völlers, Träger) ersetzt wurden. Doch die ausgeschiedenen Abgeordneten sind immer noch in den Daten enthalten! Ebenfalls wurde bei Abgeordneten, die fraktionslos geworden sind, diese Veränderung nicht eingetragen! Dieses Problem betrifft nicht nur die 19. Wahlperiode, sondern sämtliche Wahlperioden.

**So gab es z.B. in der ersten Wahlperiode nur 421 Sätze, aber 474 Einträge!**

In [34]:
len(df_WP1)

474

Es wird deutlich, dass es teilweise eine deutliche Abweichung gibt! Ein Cleaning wäre jedoch nur per Hand möglich und daher viel zu aufwendig, weshalb darauf schlicht verzichtet wird!

#### Umindizierung auf Wahlperioden

Wie bereits angekündigt, wollen wir jetzt den Index unseres DataFrames durch die Wahlperioden ersetzen, damit wir nicht länger mit kryptischen Zahlen als Index arbeiten müssen.
Hierfür müssen die einzelnen Wahlperiodendataframes mit der neuen Indexliste verknüpft werden.

Also zuerst eine einfache Liste mit einer Aufzählung von Wahlperioden erstellen (WP1, WP2, ..., WPn). Dann eine Liste mit den neuen Indexeinträgen.

In [35]:
#Einfache Aufzählungsliste

unique_list_n = []
for e in unique_list:
    unique_list_n.append("WP" + str(e))

#Der neue Index

list_years = ["1949-1953;01", "1953–1957;02","1957–1961;03", "1961–1965;04", "1965–1969;05", "1969–1972;06", "1972–1976;07","1976-1980;08",
"1980–1983;09", "1983–1987;10","1987–1990;11","1990–1994;12","1994–1998;13","1998–2002;14","2002–2005;15","2005–2009;16","2009–2013;17",
              "2013–2017;18", "2017-;19"]    

Jetzt diese einfache Liste und die Indexliste zusammen "zippen", also in einem Dictionary miteinander verknüpfen.

In [36]:
dict_WPyears = dict(zip(unique_list_n, list_years))

Einmal eine neue Liste mit allen Teildataframes bauen.

In [37]:
df_wp_list = [df_WP1, df_WP2, df_WP3, df_WP4, df_WP5, df_WP6, df_WP7,df_WP8,df_WP9, df_WP10,df_WP11,df_WP12,df_WP13,
df_WP14,df_WP15,df_WP16,df_WP17,df_WP18, df_WP19]

Und schlussendlich Funktionen definieren, die den neuen Index setzen. Die erste ändert für jedes Element der Teildataframeliste den Namen zu dem enstprechenden Index aus der unique_list_n (also anstatt df_WP1, df_WP2,... wird es 0, 1, ...). Die zweite Funktion nutzt diese Zuteilung dann, indem der neu zugeordnete Name im Dictionary gesucht wird und dann durch den jeweils passenden neuen Index ersetzt wird. Die dritte Funktion setzt diese diese neuen Namen dann auch tatsächlich als den neuen Index.

In [38]:
for d in unique_list_n:
    index = unique_list_n.index(d)
    df_wp_list[index].name = d

for e in df_wp_list:
    name = e.name
    e["WP-JAHRE"] = dict_WPyears[name]
            
for df in df_wp_list:
    df.set_index("WP-JAHRE", drop = False, inplace=True)

C:\Users\Master\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Testen wir das Ganze einmal!

In [39]:
print(df_WP1.index)
print(df_WP19["WP-JAHRE"].index)

Index(['1949-1953;01', '1949-1953;01', '1949-1953;01', '1949-1953;01',
       '1949-1953;01', '1949-1953;01', '1949-1953;01', '1949-1953;01',
       '1949-1953;01', '1949-1953;01',
       ...
       '1949-1953;01', '1949-1953;01', '1949-1953;01', '1949-1953;01',
       '1949-1953;01', '1949-1953;01', '1949-1953;01', '1949-1953;01',
       '1949-1953;01', '1949-1953;01'],
      dtype='object', name='WP-JAHRE', length=474)
Index(['2017-;19', '2017-;19', '2017-;19', '2017-;19', '2017-;19', '2017-;19',
       '2017-;19', '2017-;19', '2017-;19', '2017-;19',
       ...
       '2017-;19', '2017-;19', '2017-;19', '2017-;19', '2017-;19', '2017-;19',
       '2017-;19', '2017-;19', '2017-;19', '2017-;19'],
      dtype='object', name='WP-JAHRE', length=711)


Das sieht gut aus! Jetzt setzen wir die einzelnen Teildataframes wieder zu einem großen DataFrame zusammen und voila, bestaune den neuen Index!

In [40]:
df_indexed = pd.concat(df_wp_list)

Zur besseren Selektierung machen wir jetzt noch einen weiteren Quasi-Index NUMIND (da damit erheblich leichter zu arbeiten ist, als mit IDs ab 10.000).

In [41]:
num_index = list(range(0,11600))

df_indexed["NUMIND"] = num_index

df_indexed.set_index(["WP-JAHRE"])

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,...,WP11,WP12,WP13,WP14,WP15,WP16,WP17,WP18,WP19,NUMIND
WP-JAHRE,,,,,,,,,,,,,,,,,,,,,
1949-1953;01,11000009,1,Union,1876,Bundeskanzler a. D.,Deutschland,Adenauer,Konrad,"[1, 2, 3, 4, 5]","[1, 1, 1, 1, 1]",...,0,0,0,0,0,0,0,0,0,0
1949-1953;01,11000013,1,Andere,1904,"Bergmann, Gewerkschaftsfunktionär",Deutschland,Agatz,Willi,[1],[0],...,0,0,0,0,0,0,0,0,0,1
1949-1953;01,11000015,1,Andere,1879,Kapitän a. D.,Deutschland,Ahrens,Adolf,[1],[0],...,0,0,0,0,0,0,0,0,0,2
1949-1953;01,11000020,1,Union,1890,Geschäftsführer,Deutschland,Albers,Johannes,"[1, 2]","[1, 1]",...,0,0,0,0,0,0,0,0,0,3
1949-1953;01,11000021,0,SPD,1901,Buchhalterin,Deutschland,Albertz,Luise,"[1, 2, 3, 4, 5]","[0, 0, 0, 0, 1]",...,0,0,0,0,0,0,0,0,0,4
1949-1953;01,11000024,0,SPD,1896,Sportlehrerin,Deutschland,Albrecht,Lisa,"[1, 2, 3]","[0, 0, 0]",...,0,0,0,0,0,0,0,0,0,5
1949-1953;01,11000030,1,SPD,1889,"Journalist, Schriftsteller",Deutschland,Altmaier,Jakob,"[1, 2, 3, 4]","[1, 1, 1, 1]",...,0,0,0,0,0,0,0,0,0,6
1949-1953;01,11000032,1,Andere,1888,"Jurist (Richter), Verwaltungsbeamter",Deutschland,Amelunxen,Rudolf,[1],[0],...,0,0,0,0,0,0,0,0,0,7
1949-1953;01,11000041,0,SPD,1880,Textilarbeiterin,Deutschland,Ansorge,Maria,[1],[0],...,0,0,0,0,0,0,0,0,0,8


Und jetzt geht es noch darum das Alter entsprechend der jeweiligen Wahlperiode zu berechnen. Hierfür extrahieren wir das erste Jahr einer jeweiligen Wahlperiode, setzen dies als neue Spalte und ziehen dieses jeweilige Jahr vom Geburtsdatum ab, um das Alter zu erhalten.

In [42]:
#Zieht die ersten vier Buchstaben (also das jeweilige Startjahr) aus dem neuen Index

ANFANG_list = []
for e in df_indexed.index:
    ANFANG_list.append(int(e[:4]))

#Macht daraus eine neue Spalte

df_indexed["ANFANG"] = ANFANG_list

#Berechnet die neue Spalte "Alter" aus der Differenz
df_indexed["ALTER"] = df_indexed["ANFANG"] - df_indexed["GEBURTSDATUM"].apply(int)                      
df_indexed.head()                     

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,...,WP14,WP15,WP16,WP17,WP18,WP19,WP-JAHRE,NUMIND,ANFANG,ALTER
WP-JAHRE,,,,,,,,,,,,,,,,,,,,,
1949-1953;01,11000009,1,Union,1876,Bundeskanzler a. D.,Deutschland,Adenauer,Konrad,"[1, 2, 3, 4, 5]","[1, 1, 1, 1, 1]",...,0,0,0,0,0,0,1949-1953;01,0,1949,73
1949-1953;01,11000013,1,Andere,1904,"Bergmann, Gewerkschaftsfunktionär",Deutschland,Agatz,Willi,[1],[0],...,0,0,0,0,0,0,1949-1953;01,1,1949,45
1949-1953;01,11000015,1,Andere,1879,Kapitän a. D.,Deutschland,Ahrens,Adolf,[1],[0],...,0,0,0,0,0,0,1949-1953;01,2,1949,70
1949-1953;01,11000020,1,Union,1890,Geschäftsführer,Deutschland,Albers,Johannes,"[1, 2]","[1, 1]",...,0,0,0,0,0,0,1949-1953;01,3,1949,59
1949-1953;01,11000021,0,SPD,1901,Buchhalterin,Deutschland,Albertz,Luise,"[1, 2, 3, 4, 5]","[0, 0, 0, 0, 1]",...,0,0,0,0,0,0,1949-1953;01,4,1949,48


Eine schnelle Überprüfung auf Korrektheit der Daten.

In [43]:
df_indexed.ALTER.unique() 

array([73, 45, 70, 59, 48, 53, 60, 61, 69, 31, 55, 67, 54, 34, 56, 29, 49,
       58, 44, 36, 42, 66, 39, 43, 51, 38, 52, 37, 65, 64, 47, 50, 57, 46,
       41, 62, 40, 63, 32, 35, 68, 72, 74, 27, 33, 30, 28, 77, 75, 81, 26,
       71, 79, 25, 76, 85, 80, 89, 24, 21, 23, 22, 19, 78], dtype=int64)

Jetzt wollen wir in gleicher Weise für den neuen Index auch die Mandatsart umändern, so dass keine Liste mit Mandatsarten mehr enthalten ist, sondern für jeden Index nur noch *ein* Boolean Mandatsartwert.

In [44]:
#Erstellung einer Liste über den gesamten Quasi-Index

anumindex = list(range(0,11600))

#Neuen Index, Wahlperioden und die Mandatsarten als Listen ausgeben

awpindlst = df_indexed.index.tolist()
awplst = list(df_indexed["WAHLPERIODEN"])
amandlst = list(df_indexed["MANDATSARTEN"])
anewcol = []

Die Funktion extrahiert zunächst einfach die letzten beiden Elemente im neuen Index (also die Wahlperiodenzahl). Anschließend guckt sie, welche Zeilen des DataFrames dieselbe Zahl in ihrer "Wahlperioden"-Liste enthalten. Nun nimmt sie in genau denselben Zeilen den Subindex der jeweiligen Wahlperiodenzahl in der Liste der Wahlperioden eines Abgeordneten. Und abschließend erstellt sie eine neue Liste, die genau die Elemente dieses Subindex zum passenden Gesamtindex zuordnet.

In [45]:
for ind in anumindex:
    num = str(int(awpindlst[ind][-2:]))
    if num in awplst[ind]:
        subind = int(awplst[ind].index(num))
        anewcol.append(amandlst[ind][subind])

Kurze Überprüfung und Einfügung ins DataFrame

In [46]:
print(len(anewcol))
df_indexed["MANDATSART"] = anewcol

11600


Jetzt nochmal schnell die Mandatsart und das Alter auf None Values überprüfen und fillen.

In [47]:
df_indexed[df_indexed['ALTER'].isnull()]

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,...,WP15,WP16,WP17,WP18,WP19,WP-JAHRE,NUMIND,ANFANG,ALTER,MANDATSART
WP-JAHRE,,,,,,,,,,,,,,,,,,,,,


In [48]:
df_indexed[df_indexed['MANDATSART'].isnull()]

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,...,WP15,WP16,WP17,WP18,WP19,WP-JAHRE,NUMIND,ANFANG,ALTER,MANDATSART
WP-JAHRE,,,,,,,,,,,,,,,,,,,,,
1998–2002;14,11002710,1,Union,1955,Dipl.-Ingenieur für Schiffstechnik,Deutschland,Kuhn,Werner,"[13, 14, 15]","[1, None, 0]",...,1,0,0,0,0,1998–2002;14,8023,1998,43,NaN


In [49]:
df_indexed["MANDATSART"].fillna(0, inplace=True)
df_indexed["MANDATSART"].value_counts()
df_indexed["MANDATSART"].round(0).astype(int)
print("True")

True


Eine ganze Reihen an Spalten, die für die Analyse redundant sind, werden jetzt einfach gedropt.

In [50]:
df_indexed_final = df_indexed.drop(["ID","GEBURTSDATUM", "GEBURTSLAND", "WAHLPERIODEN","WP-JAHRE","NACHNAME", "VORNAME", "MANDATSARTEN"], axis=1)

Zum krönenden Abschluss verändern wir noch die Indexreihenfolge, um z.B. Alter nicht immer ganz am Ende stehen zu haben.

In [51]:
df_indexed_final = df_indexed_final[
['NUMIND','ANFANG', 'ALTER', 'GESCHLECHT', 'PARTEI_KURZ',"BERUF","AUSLAND", "MANDATSART", "WP1", "WP2", "WP3", "WP4", "WP5",
       "WP6", "WP7", "WP8", "WP9", "WP10", "WP11", "WP12", "WP13", "WP14",
       'WP15', 'WP16', 'WP17', 'WP18', 'WP19']]

df_indexed_final.head()

,NUMIND,ANFANG,ALTER,GESCHLECHT,PARTEI_KURZ,BERUF,AUSLAND,MANDATSART,WP1,WP2,...,WP10,WP11,WP12,WP13,WP14,WP15,WP16,WP17,WP18,WP19
WP-JAHRE,,,,,,,,,,,,,,,,,,,,,
1949-1953;01,0,1949,73,1,Union,Bundeskanzler a. D.,0,1.0,1,1,...,0,0,0,0,0,0,0,0,0,0
1949-1953;01,1,1949,45,1,Andere,"Bergmann, Gewerkschaftsfunktionär",0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1949-1953;01,2,1949,70,1,Andere,Kapitän a. D.,0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1949-1953;01,3,1949,59,1,Union,Geschäftsführer,0,1.0,1,1,...,0,0,0,0,0,0,0,0,0,0
1949-1953;01,4,1949,48,0,SPD,Buchhalterin,0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0


**Geschafft! Jetzt kann das fertige DataFrame exportiert werden.**

In [52]:
df_indexed_final.to_csv("MdB_Indexed.csv")